# Convert Yelp Dataset json files to csv

In [3]:
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from collections import defaultdict

import seaborn as sns

from pandas_profiling import ProfileReport

import pickle


In [30]:
d = defaultdict(list)

parent_dir = 'yelp_dataset/'

path_dic = {'B': parent_dir+'yelp_academic_dataset_business.json', 'C': parent_dir+'yelp_academic_dataset_checkin.json', 'R': parent_dir+'yelp_academic_dataset_review.json', 'T': parent_dir+'yelp_academic_dataset_tip.json', 'U': parent_dir+'yelp_academic_dataset_user.json'}

for key in path_dic:
    with open(path_dic[key]) as f:
        d[key] = pd.DataFrame(json.loads(line) for line in tqdm(f))


209393it [00:03, 53924.53it/s]
175187it [00:01, 107106.16it/s]
8021122it [01:09, 115464.89it/s]
1320761it [00:17, 74242.84it/s]
1968703it [01:08, 28549.65it/s]


In [4]:
business = d['B']
pickle.dump(business, open("business.p", "wb"))

In [31]:
review = pd.DataFrame(d['R'])
pickle.dump(review, open("review.p", "wb"))

In [32]:
user = pd.DataFrame(d['U'])
pickle.dump(user, open("user.p", "wb"))

In [33]:
display("business.shape: {}".format(business.shape))
display("review.shape: {}".format(review.shape))
display("user.shape: {}".format(user.shape))

'business.shape: (168903, 12)'

'review.shape: (8021122, 9)'

'user.shape: (1968703, 22)'

In [4]:
business = pickle.load(open('business.p', 'rb'))
review = pickle.load(open('review.p', 'rb'))
user = pickle.load(open('user.p', 'rb'))

# Keep business that are open in the PA

In [5]:
business.is_open.value_counts()

1    168903
0     40490
Name: is_open, dtype: int64

In [6]:
business = business[business['is_open']==1].drop(['hours', 'is_open'], axis=1)

In [13]:
business_us = business[business.state=='PA']

# Remove NAs

In [14]:
def rename_removena(df, string):
    df = df.dropna()
    new_columns = string + df.columns.values
    df.columns = new_columns

    return df

In [15]:
business_us = rename_removena(business_us, 'business_')
business_us.rename(columns={'business_business_id':'business_id'}, inplace=True)

review = rename_removena(review, 'review_')
review.rename(columns={'review_review_id':'review_id', 'review_user_id':'user_id', 'review_business_id':'business_id'}, inplace=True)

user = rename_removena(user, 'user_')
user.rename(columns={'user_user_id':'user_id'}, inplace=True)



# Look at only restaurants

In [16]:
business_us_explode = business_us.assign(business_categories = business_us.business_categories.str.split(', ')).explode('business_categories')

In [17]:
print('Top 10 categories:')
business_us_explode.business_categories.value_counts()[:30]

Top 10 categories:


Restaurants                  2962
Food                         1685
Shopping                     1653
Beauty & Spas                 905
Nightlife                     816
Home Services                 792
Bars                          755
Health & Medical              681
Local Services                660
Automotive                    631
Pizza                         602
Event Planning & Services     591
American (Traditional)        548
Sandwiches                    520
Active Life                   504
Home & Garden                 386
American (New)                383
Coffee & Tea                  376
Fast Food                     365
Fashion                       361
Hair Salons                   346
Arts & Entertainment          339
Italian                       333
Burgers                       327
Auto Repair                   324
Breakfast & Brunch            305
Nail Salons                   302
Fitness & Instruction         234
Hotels & Travel               222
Grocery       

In [54]:
business_us_explode[business_us_explode['business_categories'].str.contains('american', case=False)].business_categories.value_counts()

American (Traditional)    4377
American (New)            2834
Latin American             281
Name: business_categories, dtype: int64

In [18]:
business_us_food = business_us[business_us['business_categories'].str.contains('restaurants|food', case=False)]

In [19]:
business_us_food = business_us_food[~business_us_food['business_categories'].str.contains('fast food', case=False)]

# Merge review and business tables

In [20]:
df = pd.merge(business_us_food, review, on='business_id', how='inner')
df = pd.merge(df, user, how='inner', on='user_id')
display("df.shape: {}".format(df.shape))
pickle.dump(df, open("df.p", "wb"))

'df.shape: (222293, 41)'